In [1]:
!pip install timm sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sh: filename=sh-1.14.3-py2.py3-none-any.whl size=39656 sha256=8dbddb376a17774c834b2d59e102c3d9a19d86f2a7dff735d72b747ce451adbe
  Stored in directory: /root/.cache/pip/wheels/e0/d2/e1/3290bef6382cfc1ec5e659353e5f403fc1eb385ea5f615519a
Successfully built sh


In [2]:
import pandas as pd
import sh
import os
import torch
import timm
from torchvision.transforms.functional import resize, normalize, pil_to_tensor
from datasets import load_dataset
from tqdm.notebook import tqdm

Copy data into an appropriate directory structure to be parsed by imagefolder.

In [3]:
furniture = pd.read_csv('../input/furniture-images-dataset/furniture_data_img.csv')
furniture_type_to_class = {type: f'furniture_{cls}' for cls, type in enumerate(furniture.Furniture_Type.unique())}
furniture_groups = furniture.groupby('Furniture_Type')
for furniture_type, group in tqdm(furniture_groups):
    cls = furniture_type_to_class[furniture_type]
    sh.mkdir('-p', f'train/{cls}')
    sh.mkdir('-p', f'val/{cls}')
    for i, example_file in enumerate(tqdm(group.Image_File)):
        if i / len(group.Image_File) < 0.9:
            split = 'train'
        else:
            split = 'val'
        src = '../input/furniture-images-dataset/furniture_images' + example_file
        tgt = f'{split}/{cls}/{os.path.basename(example_file)}'
        if not os.path.isfile(tgt):
            sh.cp(src, tgt)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/209 [00:00<?, ?it/s]

  0%|          | 0/3043 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/1813 [00:00<?, ?it/s]

  0%|          | 0/1326 [00:00<?, ?it/s]

  0%|          | 0/244 [00:00<?, ?it/s]

  0%|          | 0/2419 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Compute BEiT embeddings, store and dump dataset to disk.

In [4]:
train_dataset = load_dataset('imagefolder', data_dir='train')['train']
train_dataset

Resolving data files:   0%|          | 0/8415 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset image_folder downloaded and prepared to /root/.cache/huggingface/datasets/image_folder/default-55b0dccca0484bfd/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 8415
})

In [5]:
val_dataset = load_dataset('imagefolder', data_dir='val')['train']
val_dataset

Resolving data files:   0%|          | 0/931 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset image_folder downloaded and prepared to /root/.cache/huggingface/datasets/image_folder/default-5c5f5b960f3fde2e/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 931
})

In [6]:
torch.cuda.is_available()

True

In [7]:
device = torch.device('cuda')

def compute_embedding(batch):
    tensors = [pil_to_tensor(x.convert('RGB')) for x in batch['image']]
    tensors = [resize(tensor / 255., size=(224, 224)) for tensor in tensors]
    tensors = [normalize(tensor, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) for tensor in tensors]
    tensors = torch.stack(tensors)
    tensors = tensors.to(device=device)
    with torch.no_grad():
        batch['beit'] = beit(tensors).cpu().numpy()
    return batch

beit = timm.create_model(
    'beit_large_patch16_224_in22k', pretrained=True, num_classes=0
).to(device=device)
train_dataset = train_dataset.map(compute_embedding, batched=True)
val_dataset = val_dataset.map(compute_embedding, batched=True)
train_dataset = train_dataset.remove_columns('image')
val_dataest = val_dataset.remove_columns('image')
train_dataset.save_to_disk('furniture_train')
val_dataset.save_to_disk('furniture_val')

/opt/conda/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:2227.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://conversationhub.blob.core.windows.net/beit-share-public/beit/beit_large_patch16_224_pt22k_ft22k.pth" to /root/.cache/torch/hub/checkpoints/beit_large_patch16_224_pt22k_ft22k.pth


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Remove raw images from output.

In [8]:
sh.rm('-rf', 'train', 'val')